In [119]:
import os
import cv2 # OpenCV ou cv2 para tratamento de imagens;
import numpy as np # Numpy para trabalharmos com matrizes n-dimensionais
from keras.models import Sequential # Importando modelo sequencial
from keras.layers.convolutional import Conv2D, MaxPooling2D # Camada de convolução e max pooling
from keras.layers.core import Activation, Flatten, Dense, Dropout # Camada da função de ativação, flatten, entre outros
from keras import backend as K # backend do keras
from keras.optimizers import Adam # optimizador Adam
from keras.utils import to_categorical # Função utilizada para categorizar listas de treino e teste
from keras.preprocessing.image import ImageDataGenerator # Classe para ajudar na variação de amostras de treinamento
from keras.callbacks import ModelCheckpoint # Classe utilizada para acompanhamento durante o treinamento onde definimos os atributos que serão considerados para avaliação
import matplotlib.pyplot as plt
import tensorflow as tf

In [120]:
%cd ..


/


In [121]:
from utils import get_data

In [ ]:
data_path = '/Users/rodrigo/Documents/SPR_X_data/spr-x-ray-age/kaggle/kaggle/train/'
label_data_path = '/Users/rodrigo/Documents/SPR_X_data/spr-x-ray-age/train_gender.csv'
width = 500
height = 500
final_data_path = None
input_shape = (500,500,1)


X, y = get_data(data_path, label_data_path, width, height)

In [ ]:
plt.imshow(X[-104], cmap='Greys_r')

In [ ]:
X_train = X[:-1702]
y_train = y[:-1702]
X_val = X[-1720:]
y_val = y[-1720:]

In [ ]:
y_val.sum()/len(y_val)

In [ ]:
strategy = tf.test.gpu_device_name()
if len(strategy) > 0:
    print("Found GPU at: {}".format(strategy))
else:
    strategy = "/device:CPU:0"
    print("No GPU, using {}.".format(strategy))

In [ ]:

def create_lenet(input_shape):

    # Definimos que estamos criando um modelo sequencial
    model = Sequential()

    # Primeira camada do modelo:
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=input_shape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Segunda camada do modelo:
    model.add(Conv2D(64, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Primeira camada fully connected
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))

    # Classificador softmax
    model.add(Dense(classes))
    model.add(Activation("Softmax"))
    return model

def CNN(input_shape):
    cnn = Sequential()
    
    cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=input_shape, strides=(2,2)))
    cnn.add(MaxPooling2D(pool_size = (2, 2)))
    cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=input_shape))
    cnn.add(MaxPooling2D(pool_size = (2, 2)))
    cnn.add(Dropout(0.3))
    cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=input_shape))
    cnn.add(MaxPooling2D(pool_size = (2, 2)))
    cnn.add(Conv2D(64, (3, 3), activation="relu", input_shape=input_shape))
    cnn.add(MaxPooling2D(pool_size = (2, 2)))
    cnn.add(Dropout(0.3))
    cnn.add(Conv2D(64, (3, 3), activation="relu", input_shape=input_shape))
    cnn.add(MaxPooling2D(pool_size = (2, 2)))
    
    cnn.add(Flatten())
    cnn.add(Dense(activation = 'relu', units = 64))
    cnn.add(Dropout(0.3))
    cnn.add(Dense(activation = 'relu', units = 32))
    cnn.add(Dense(activation = 'sigmoid', units = 1))
    return cnn

if __name__ == '__main__':
    
    #models_path = "models" # Defina aqui onde serão salvos os modelos na sua maquina
    classes = 2 # Quantidade de classes que o modelo utilizará
    epochs = 10 # Quantidade de épocas (a quantidade de iterações que o modelo realizará durante o treinamento)
    init_lr = 1e-3 # Taxa de aprendizado a ser utilizado pelo optimizador
    batch_size = 32 # Tamanho dos lotes utilizados por cada epoca
    
#     save_model = os.path.join(models_path, "lenet-{epoch:02d}-{acc:.3f}-{val_acc:.3f}.model")
#     os.makedirs(models_path, exist_ok=True)



    aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                fill_mode="nearest")

    model = CNN(input_shape)

    opt = Adam(lr=init_lr, decay=init_lr / epochs)

    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
    model.summary()
    
    history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_val, y_val))

    print("\n training network")

#     checkpoint1 = ModelCheckpoint(save_model, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#     checkpoint2 = ModelCheckpoint(save_model, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#     callbacks_list = [checkpoint1,checkpoint2]

#     H = model.fit_generator(aug.flow(X_train, y_train, batch_size=batch_size),
#             validation_data=(X_val, y_val), steps_per_epoch=len(X_train) // batch_size,
#             epochs=epochs, verbose=1)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

#test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)